<a href="https://colab.research.google.com/github/dipnarayan501/Deep_learning/blob/main/Assignment1_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Importing libraries

import numpy as np
import pandas as pd
from tensorflow.keras.datasets import mnist, fashion_mnist
from sklearn.model_selection import train_test_split

In [5]:
# Preparing Dataset

def prepare_dataset():
    (X, y), (X_test, y_test) = fashion_mnist.load_data()

    # Reshaping the data matrices
    X = X.reshape(X.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)

    # Normalizing the pixel intensities
    X = X/255.0
    X_test = X_test/255.0
    
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
    
    y_train_encode = np.zeros((10,y_train.shape[0]))
    y_train_encode[y_train, np.array(list(range(y_train.shape[0])))] = 1

    # Split the X_train into a training set and validation set
    
    
    print("Number of examples in training set")
    print(X_train.shape)
    print(y_train_encode.shape)
    print(y_train.shape)
    
    print("No of examples in validation set")
    print(X_val.shape)
    print(y_val.shape)
    print("No of examples in test set")
    print(X_test.shape)
    print(y_test.shape)
    
    return X_train.T, y_train_encode.T, y_train.T ,X_val.T, y_val.T, X_test.T, y_test.T


X_train, y_train_encode, y_train ,X_val, y_val, X_test, y_test = prepare_dataset()

4431872/4422102 [==============================] - 0s 0us/step
Number of examples in training set
(54000, 784)
(10, 54000)
(54000,)
No of examples in validation set
(6000, 784)
(6000,)
No of examples in test set
(10000, 784)
(10000,)


In [6]:
#Activation functions

def sigmoid(x):
    return 1. / (1.+np.exp(-x))
def relu(x):
    return np.maximum(0,x)
def tanh(x):
    return np.tanh(x)
def softmax(x):
    soft = np.zeros(x.shape)
    for i in range(0, x.shape[1]):
        numr = np.exp(x[:, i])
        soft[:, i] = numr/np.sum(numr)
    return soft



In [7]:
#Initializing parameters
output_layer = y_test.shape[0]
input_layer = X_test.shape[0]
hidden_layer = [64, 64]
#Example  28*28,[64,64],output_classes
def initialize_parameters(input_layer, hidden_layer, output_layer,mode):
    weights = []
    biases  = []
    layers = [input_layer] + hidden_layer + [output_layer]
    for i in range(len(hidden_layer)+1):
        if mode == 'random_normal':
            weights.append(np.random.randn(layers[i+1], layer_dims[i])) * 0.01
        if mode == 'xavier':
            weights.append(np.random.randn(layers[i+1],layers[i])*np.sqrt(2/layers[i+1]))
        biases.append(np.random.randn(layers[i+1],1))
    return weights, biases
weights, biases = initialize_parameters(input_layer, hidden_layer, output_layer, 'xavier')
for i in range(len(weights)):
    print(weights[i].shape)
    
for i in range(len(biases)):
    print(biases[i].shape)


(64, 784)
(64, 64)
(10000, 64)
(64, 1)
(64, 1)
(10000, 1)
